In [62]:
import requests, json, pandas as pd, time

In [2]:
payload = {'draw': '1', 'start': 0, 'length': 5}
headers = {'Cookie': 'sample_result_sample_id=25; threshold=all'}
r = requests.get('http://www.licpathway.net/ATACdb/search/search_sample_table.php', params=payload, headers=headers)

In [13]:
response_data = r.json()
print(r.text)

{"draw":"1","recordsTotal":"48025","recordsFiltered":"48025","data":[{"car_chr":"chr1","car_start":"9997","car_end":"10802","car_size":"805","car_intlogqvalue":"1359","car_foldchange":"13.36316","car_pvalue":"138.88042","car_qvalue":"135.96225","car_summit_to_start":"204","car_sample_id":"25","car_region_id":"1","car_gsm_id":"SRR6955601","car_risk_snp_anno":"0","car_eqtl_anno":"0","car_crisps_anno":"0","car_te_anno":"14","car_se_anno":"0","car_methylation_anno":"0","car_interaction_anno":"1","car_commmon_snp_anno":"5"},{"car_chr":"chr1","car_start":"20963","car_end":"21340","car_size":"377","car_intlogqvalue":"108","car_foldchange":"5.73316","car_pvalue":"13.03911","car_qvalue":"10.84643","car_summit_to_start":"329","car_sample_id":"25","car_region_id":"2","car_gsm_id":"SRR6955601","car_risk_snp_anno":"0","car_eqtl_anno":"0","car_crisps_anno":"0","car_te_anno":"0","car_se_anno":"0","car_methylation_anno":"0","car_interaction_anno":"0","car_commmon_snp_anno":"0"},{"car_chr":"chr1","car_

In [12]:
rows = response_data['data']
for row in rows:
    print(json.dumps(row, indent=4))

{
    "car_chr": "chr1",
    "car_start": "9997",
    "car_end": "10802",
    "car_size": "805",
    "car_intlogqvalue": "1359",
    "car_foldchange": "13.36316",
    "car_pvalue": "138.88042",
    "car_qvalue": "135.96225",
    "car_summit_to_start": "204",
    "car_sample_id": "25",
    "car_region_id": "1",
    "car_gsm_id": "SRR6955601",
    "car_risk_snp_anno": "0",
    "car_eqtl_anno": "0",
    "car_crisps_anno": "0",
    "car_te_anno": "14",
    "car_se_anno": "0",
    "car_methylation_anno": "0",
    "car_interaction_anno": "1",
    "car_commmon_snp_anno": "5"
}
{
    "car_chr": "chr1",
    "car_start": "20963",
    "car_end": "21340",
    "car_size": "377",
    "car_intlogqvalue": "108",
    "car_foldchange": "5.73316",
    "car_pvalue": "13.03911",
    "car_qvalue": "10.84643",
    "car_summit_to_start": "329",
    "car_sample_id": "25",
    "car_region_id": "2",
    "car_gsm_id": "SRR6955601",
    "car_risk_snp_anno": "0",
    "car_eqtl_anno": "0",
    "car_crisps_anno": "0"

In [17]:
def grab_some_data(sample_id, start, length, draw):
    payload = {'draw': draw, 'start': start, 'length': length}
    headers = {'Cookie': 'sample_result_sample_id=' + sample_id + '; threshold=all'}
    r = requests.get('http://www.licpathway.net/ATACdb/search/search_sample_table.php', params=payload, headers=headers)
    response_data = r.json()
    return response_data['data'], int(response_data['recordsTotal'])

In [29]:
def get_records_loop_support(data_container, sample_id, s, l, draw, counter):
    data, total_records = grab_some_data(sample_id, s, l, draw)
    data_container.extend(data)
    #print(f'got page {counter}')

    s += l
    draw += 1
    counter += 1
    
    return total_records, s, draw, counter

In [30]:
def get_all_for_sample(sample_id):
    s = 0
    l = 1000
    draw = 1
    counter = 1

    sample_data = []

    total_records, s, draw, counter = get_records_loop_support(sample_data, sample_id, s, l, draw, counter)


    while s < total_records:
        _, s, draw, counter = get_records_loop_support(sample_data, sample_id, s, l, draw, counter)
        
    return sample_data

In [45]:
sample_25_id = '25'
sample_25_data = get_all_for_sample(sample_25_id)

In [46]:
print(len(sample_25_data))

48025


In [47]:
print(sample_25_data[47960:47980])

[{'car_chr': 'chrX', 'car_start': '154299083', 'car_end': '154300294', 'car_size': '1211', 'car_intlogqvalue': '1151', 'car_foldchange': '12.33662', 'car_pvalue': '118.03798', 'car_qvalue': '115.19642', 'car_summit_to_start': '833', 'car_sample_id': '25', 'car_region_id': '47961', 'car_gsm_id': 'SRR6955601', 'car_risk_snp_anno': '0', 'car_eqtl_anno': '0', 'car_crisps_anno': '0', 'car_te_anno': '85', 'car_se_anno': '0', 'car_methylation_anno': '630', 'car_interaction_anno': '822', 'car_commmon_snp_anno': '5'}, {'car_chr': 'chrX', 'car_start': '154417161', 'car_end': '154417425', 'car_size': '264', 'car_intlogqvalue': '202', 'car_foldchange': '9.42108', 'car_pvalue': '22.60504', 'car_qvalue': '20.28737', 'car_summit_to_start': '82', 'car_sample_id': '25', 'car_region_id': '47962', 'car_gsm_id': 'SRR6955601', 'car_risk_snp_anno': '0', 'car_eqtl_anno': '0', 'car_crisps_anno': '0', 'car_te_anno': '14', 'car_se_anno': '0', 'car_methylation_anno': '0', 'car_interaction_anno': '95', 'car_commm

In [50]:
def process_sample_data(sample_data, sample_name):
    df = pd.read_json(json.dumps(sample_data))
    #display(df)
    df.to_csv(f'sample-{sample_name}.csv')

In [51]:
process_sample_data(sample_25_data, sample_25_id)

In [52]:
samples_frame = pd.read_csv('Samples-Of-Interest.csv')

In [53]:
display(samples_frame)

,Sample ID,Biosample type,Biosample name,Tissue type,Cancer type,Region number
0,Sample_0025,Primary Cell,CD56Bright natural killer cell,Blood,Normal,48025
1,Sample_0026,Primary Cell,CD56Bright natural killer cell,Blood,Normal,50447
2,Sample_0027,Primary Cell,CD56Bright natural killer cell,Blood,Normal,47349
3,Sample_0028,Primary Cell,CD56Bright natural killer cell,Blood,Normal,46698
4,Sample_0029,Primary Cell,CD56Dim natural killer cell,Blood,Normal,40745
...,...,...,...,...,...,...
447,Sample_1467,Other,Macrophage,Blood,Other,10568
448,Sample_1468,Other,Monocyte,Blood,Other,6102
449,Sample_1469,Other,Monocyte,Blood,Other,8042
450,Sample_1470,Other,Neutrophile,Blood,Other,7806


In [58]:
samples_json = json.loads(samples_frame['Sample ID'].to_json())

In [61]:
list_of_samples = [int(v[7:]) for k,v in samples_json.items()]
print(list_of_samples)

[25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 93, 94, 95, 96, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 128, 129, 130, 131, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362, 363, 447, 543, 544, 584, 585, 586, 587, 614, 615

In [63]:
for sample_number in list_of_samples:
    sample_id = str(sample_number)
    sample_data = get_all_for_sample(sample_id)
    
    process_sample_data(sample_data, sample_id)
    print(f'Saved data for sample {sample_id}')
    time.sleep(5)

Saved data for sample 25
Saved data for sample 26
Saved data for sample 27
Saved data for sample 28
Saved data for sample 29
Saved data for sample 30
Saved data for sample 31
Saved data for sample 32
Saved data for sample 33
Saved data for sample 34
Saved data for sample 93
Saved data for sample 94
Saved data for sample 95
Saved data for sample 96
Saved data for sample 104
Saved data for sample 105
Saved data for sample 106
Saved data for sample 107
Saved data for sample 108
Saved data for sample 109
Saved data for sample 110
Saved data for sample 111
Saved data for sample 112
Saved data for sample 113
Saved data for sample 114
Saved data for sample 128
Saved data for sample 129
Saved data for sample 130
Saved data for sample 131
Saved data for sample 199
Saved data for sample 200
Saved data for sample 201
Saved data for sample 202
Saved data for sample 203
Saved data for sample 204
Saved data for sample 205
Saved data for sample 206
Saved data for sample 207
Saved data for sample 208
